In [4]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 31292227584 (29.14 GB)
DFS Remaining: 31292178432 (29.14 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.20.0.3:9866 (project-4-p4-raffanti-dn-1.project-4-p4-raffanti_default)
Hostname: a5564c2141ff
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 10158178304 (9.46 GB)
DFS Remaining: 15646072832 (14.57 GB)
DFS Used%: 0.00%
DFS Remaining%: 60.5

In [5]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

In [6]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [7]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/single.csv
!hdfs dfs -du -h hdfs://boss:9000/double.csv

166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [8]:
import requests
fileStat = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS&offset=0&length=200&noredirect=true")
fileStat.raise_for_status
fileStat.content

b'{"FileStatus":{"accessTime":1710867792730,"blockSize":1048576,"childrenNum":0,"fileId":16386,"group":"supergroup","length":174944099,"modificationTime":1710867798021,"owner":"root","pathSuffix":"","permission":"644","replication":1,"storagePolicy":0,"type":"FILE"}}'

In [10]:
#q3
fileStat.json()

{'FileStatus': {'accessTime': 1710867792730,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1710867798021,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [11]:
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.content

b'{"Location":"http://a5564c2141ff:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0"}'

In [12]:
#q4
r.json()["Location"]

'http://a5564c2141ff:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [13]:
file_status = fileStat.json()
file_length = file_status['FileStatus']['length']
block_size = file_status['FileStatus']['blockSize']
total_blocks = -(-file_length // block_size)  # Ceil division to get total blocks

container_block_count = {}

# Loop through each block to extract container IDs
for block_id in range(total_blocks):
    block_url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={block_id * block_size}&noredirect=true"

    response = requests.get(block_url)
    location = response.json()

    block_location = location['Location']
    container_id = block_location.split('/')[2]

    if container_id in container_block_count:
        container_block_count[container_id] += 1
    else:
        container_block_count[container_id] = 1

In [14]:
#q5
container_block_count

{'a5564c2141ff:9864': 79, 'ff1da0dcff11:9864': 88}

In [15]:
import pyarrow as pa
import pyarrow.fs
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

2024-03-19 17:05:09,689 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
with hdfs.open_input_file("/single.csv") as f:
    firstTen = f.read_at(10, 0)

In [17]:
#q6
firstTen

b'activity_y'

In [18]:
import io

count = 0
strToSearch = "Single Family"
with hdfs.open_input_file("hdfs://boss:9000/single.csv") as file:
    reader = io.TextIOWrapper(io.BufferedReader(file))
    for line in reader:
        if not line:
            break
        if strToSearch in line:
            count += 1

In [19]:
#q7
count

444874